# This file details the step by step analysis of the russian asat

In [1]:
from pathlib import Path
HERE = Path('.').resolve()
OUTPUT = HERE / 'output' / 'russian_asat'
OUTPUT.mkdir(exist_ok=True)
print(f'Using {OUTPUT} as output')

Using /home/danielk/git/antisat_analysis/projects/output/russian_asat as output


In [2]:
import sys
print(f'Adding {str(HERE.parent)} to path')
sys.path.append(str(HERE.parent))

Adding /home/danielk/git/antisat_analysis to path


In [3]:
spade_data_paths = {
    'uhf': [
        '/home/danielk/data/spade/beamparks/uhf/2021.11.23',
        '/home/danielk/data/spade/beamparks/uhf/2021.11.25',
        '/home/danielk/data/spade/beamparks/uhf/2021.11.29',
    ],
    'esr': [
        '/home/danielk/data/spade/beamparks/esr/2021.11.19',
        '/home/danielk/data/spade/beamparks/esr/2021.11.23',
        '/home/danielk/data/spade/beamparks/esr/2021.11.25',
        '/home/danielk/data/spade/beamparks/esr/2021.11.29',
    ]
}

data_paths = {}
for key in spade_data_paths:
    data_paths[key] = []
    for pth in spade_data_paths[key]:
        data_paths[key].append(OUTPUT / key / (Path(pth).name + '.h5'))

## Convert event.txt to h5

In [4]:
from convert_spade_events_to_h5 import main as conv_main

for key in spade_data_paths:
    for pth, out in zip(spade_data_paths[key], data_paths[key]):

        pth = Path(pth).resolve()
        if not out.parent.is_dir():
            out.parent.mkdir(parents=True, exist_ok=True)
        if not out.is_file():
            cmd = f'{str(pth)} {str(out)}'
            print(f'Running {cmd}')
            conv_main(cmd.split())
        else:
            print(f'{out} exsists, skipping...')


/home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.29.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.19.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.23.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.25.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.29.h5 exsists, skipping...


In [5]:
from astropy.time import Time, TimeDelta
import h5py
import numpy as np

# Determine dates of files
dates = []
dates_keyed = {}
for key in data_paths:
    dates_keyed[key] = []
    for pth in data_paths[key]:
        with h5py.File(pth, 'r') as hf:
            epoch = Time(np.min(hf['t']), format='unix', scale='utc')
            dates.append(epoch.datetime.date())
            dates_keyed[key].append(epoch.datetime.date())

dates = np.unique(dates).tolist()

In [6]:
for dt in dates:
    print(dt.strftime('%Y-%m-%d'))

2021-11-19
2021-11-23
2021-11-25
2021-11-29


## Download spacetrack catalog

Run the commands generate below to get the appropriate catalogs

In [7]:
# Set this to None if you dont want to use a secret-tool to get credentials
#  they need to be stored under the "password" and "username" attributes
secret_tool_key = 'space-track'

catalog_commands = []
tle_catalogs = []
for dt in dates:
    dt_str = dt.strftime('%Y-%m-%d')
    
    out_tle = OUTPUT / f'{dt_str.replace("-","_")}_spacetrack.tle'
    tle_catalogs.append(out_tle)
    
    cmd = f'./space_track_download.py 24h {dt_str} {str(out_tle)}'
    if secret_tool_key is not None:
        cmd += f' -k {secret_tool_key}'
    
    print(cmd)
    catalog_commands.append(cmd)

./space_track_download.py 24h 2021-11-19 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_19_spacetrack.tle -k space-track
./space_track_download.py 24h 2021-11-23 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack.tle -k space-track
./space_track_download.py 24h 2021-11-25 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack.tle -k space-track
./space_track_download.py 24h 2021-11-29 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack.tle -k space-track


In [8]:
#Verify that catalogs exist
for pth in tle_catalogs:
    assert pth.is_file(), f'{pth} does not exist'
    print(f'{pth} exists...')

/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_19_spacetrack.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack.tle exists...


## Correlate measurnments

### Calculate correlations

Choose number of cores to run on

In [15]:
cores = 6
override = False
jitter = True
r_scaling = 1.0
dr_scaling = 0.2
save_states = False

In [18]:
correlate_commands = []

override_cm = '-c' if override else ''
jitter_cm = '--jitter' if jitter else '' --save-states
save_states_cm = '--save-states' if save_states else '' 
r_scaling_cm = f'--range-scaling {r_scaling}'
dr_scaling_cm = f'--range-rate-scaling {dr_scaling}'

correlations = {}
for key in data_paths:
    correlations[key] = []
    for pth, dt in zip(data_paths[key], dates_keyed[key]):
        tle_file = tle_catalogs[dates.index(dt)]
        out_file = OUTPUT / (pth.stem + f'_{key}_correlation.pickle')
        
        correlations[key].append(out_file)
        
        cmd = f'mpirun -n {cores} ./beampark_correlator.py eiscat_{key} '
        cmd += f'{str(tle_file)} {str(pth)} {str(out_file)} {override_cm} '
        cmd += f'{jitter_cm} {r_scaling_cm} {dr_scaling_cm} {save_states_cm} -c'
        print(cmd + '\n')
        correlate_commands.append(cmd)



mpirun -n 6 ./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23.h5 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_uhf_correlation.pickle  --jitter --range-scaling 1.0 --range-rate-scaling 0.2  -c

mpirun -n 6 ./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_correlation.pickle  --jitter --range-scaling 1.0 --range-rate-scaling 0.2  -c

mpirun -n 6 ./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.29.h5 /home/danielk/git/antisa

In [50]:
#Verify that catalogs exist
for key in correlations:
    for pth in correlations[key]:
        assert pth.is_file(), f'{pth} does not exist'
        print(f'{pth} exists...')

/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_uhf_correlation.pickle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_correlation.pickle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_uhf_correlation.pickle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.19_esr_correlation.pickle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_esr_correlation.pickle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_esr_correlation.pickle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_esr_correlation.pickle exists...


### Analyse correalations

Set `threshold` to `None` to auto determine threshold. The unit of threshold is determined by the scaling that is applied to range in meter and range rate in meters per second.

In [62]:
threshold = None #1.0e3

In [63]:
correlation_analysis_cmds = []

threshold_cm = '' if threshold is None else f'--threshold {threshold}'

correlation_select = {}
for key in data_paths:
    correlation_select[key] = []
    for pth, cor in zip(data_paths[key], correlations[key]):
        out_folder = OUTPUT / (pth.stem + f'_{key}_correlation_plots')
        
        correlation_select[key].append(out_folder)
        
        cmd = f'./beampark_correlation_analysis.py --output {out_folder} '
        cmd += f'{threshold_cm} {r_scaling_cm} {dr_scaling_cm} '
        cmd += f'{cor} {pth}'
        
        print(cmd + '\n')
        correlation_analysis_cmds.append(cmd)


./beampark_correlation_analysis.py --output /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_uhf_correlation_plots  --range-scaling 1.0 --range-rate-scaling 0.2 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_uhf_correlation.pickle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23.h5

./beampark_correlation_analysis.py --output /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_correlation_plots  --range-scaling 1.0 --range-rate-scaling 0.2 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_correlation.pickle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5

./beampark_correlation_analysis.py --output /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_uhf_correlation_plots  --range-scaling 1.0 --range-rate-scaling 0.2 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_uh

In [64]:
print(' && '.join(correlation_analysis_cmds))

./beampark_correlation_analysis.py --output /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_uhf_correlation_plots  --range-scaling 1.0 --range-rate-scaling 0.2 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_uhf_correlation.pickle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23.h5 && ./beampark_correlation_analysis.py --output /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_correlation_plots  --range-scaling 1.0 --range-rate-scaling 0.2 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_correlation.pickle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5 && ./beampark_correlation_analysis.py --output /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_uhf_correlation_plots  --range-scaling 1.0 --range-rate-scaling 0.2 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.2

In [37]:
for key in correlation_select:
    paths = []
    for pth in correlation_select[key]:
        if not pth.is_file():
            paths.append(list(pth.glob('*.npy'))[0])
        else:
            paths.append(pth)
    correlation_select[key] = paths

## Plot measurements & debris cloud

Hand-craft catalogs for kosmos debris

In [53]:
meas_plot_cmds = []
for key in data_paths:
    for pth in data_paths[key]:
        out_folder = OUTPUT / (pth.stem + f'_{key}_measurement_plots')
        out_folder.mkdir(exist_ok=True)
        
        cmd = f'./beampark_plot.py -o {out_folder} -r "{key}" {pth}'
        meas_plot_cmds.append(cmd)
        
print(' && '.join(meas_plot_cmds))

./beampark_plot.py -o /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_uhf_measurement_plots -r "uhf" /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23.h5 && ./beampark_plot.py -o /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_measurement_plots -r "uhf" /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5 && ./beampark_plot.py -o /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_uhf_measurement_plots -r "uhf" /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.29.h5 && ./beampark_plot.py -o /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.19_esr_measurement_plots -r "esr" /home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.19.h5 && ./beampark_plot.py -o /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_esr_measurement_plots -r "esr" /home/danielk/git/an

### Download kosmos TLEs

In [48]:
import datetime

dt_max = datetime.timedelta(days=2)
back = f'{(dt_max.days+1)*24}h'

kosmos_catalogs = []
kosmos_download_cmds = []
for dt in dates:
    outf = OUTPUT / f'{str(dt).replace("-", "_")}_spacetrack_kosmos.tle'
    kosmos_catalogs.append(outf)
    cmd = f'./space_track_download.py {back} {str(dt + dt_max)} '
    cmd += f'{outf} -k space-track -n "COSMOS 1408 DEB"'
    print('\n' + cmd)
    kosmos_download_cmds.append(cmd)


./space_track_download.py 72h 2021-11-21 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_19_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB"

./space_track_download.py 72h 2021-11-25 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB"

./space_track_download.py 72h 2021-11-27 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB"

./space_track_download.py 72h 2021-12-01 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB"


In [49]:
' && '.join(kosmos_download_cmds)

'./space_track_download.py 72h 2021-11-21 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_19_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB" && ./space_track_download.py 72h 2021-11-25 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB" && ./space_track_download.py 72h 2021-11-27 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB" && ./space_track_download.py 72h 2021-12-01 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack_kosmos.tle -k space-track -n "COSMOS 1408 DEB"'

In [51]:
for pth in kosmos_catalogs:
    assert pth.is_file(), f'{pth} does not exist'
    print(f'{pth} exists...')

/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_19_spacetrack_kosmos.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack_kosmos.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack_kosmos.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack_kosmos.tle exists...


### Correlate with kosmos debs

In [69]:
kosmos_threshold = 3e3
kosmos_threshold_cm = '' if kosmos_threshold is None else f'--threshold {kosmos_threshold}'

In [70]:
kosmos_correlate_commands = []
kosmos_correlations = {}
for key in data_paths:
    kosmos_correlations[key] = []
    for pth, dt in zip(data_paths[key], dates_keyed[key]):
        tle_file = kosmos_catalogs[dates.index(dt)]
        if tle_file.stat().st_size == 0:
            kosmos_correlations[key].append(None)
            print('no kosmos elements, skipping...')
            continue
        out_file = OUTPUT / (pth.stem + f'_{key}_kosmos_correlation.pickle')
        
        kosmos_correlations[key].append(out_file)
        
        cmd = f'./beampark_correlator.py eiscat_{key} '
        cmd += f'{str(tle_file)} {str(pth)} {str(out_file)} {override_cm} '
        cmd += f'--target-epoch {dt} '
        cmd += f'{jitter_cm} {r_scaling_cm} {dr_scaling_cm} {save_states_cm} -c'
        kosmos_correlate_commands.append(cmd)

print(' && '.join(kosmos_correlate_commands))
print('\n')  

kosmos_correlation_analysis_cmds = []
kosmos_correlation_select = {}
for key in data_paths:
    kosmos_correlation_select[key] = []
    for pth, cor in zip(data_paths[key], kosmos_correlations[key]):
        if cor is None:
            print('no kosmos correlation, skipping...')
            kosmos_correlation_select[key].append(None)
            continue
        out_folder = OUTPUT / (pth.stem + f'_{key}_kosmos_correlation_plots')
        
        kosmos_correlation_select[key].append(out_folder)
        
        cmd = f'./beampark_correlation_analysis.py --output {out_folder} '
        cmd += f'{kosmos_threshold_cm} {r_scaling_cm} {dr_scaling_cm} '
        cmd += f'{cor} {pth}'
        
        kosmos_correlation_analysis_cmds.append(cmd)
print(' && '.join(kosmos_correlation_analysis_cmds))

no kosmos elements, skipping...
no kosmos elements, skipping...
no kosmos elements, skipping...
./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack_kosmos.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_uhf_kosmos_correlation.pickle  --target-epoch 2021-11-25 --jitter --range-scaling 1.0 --range-rate-scaling 0.2  -c && ./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack_kosmos.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.29.h5 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_uhf_kosmos_correlation.pickle  --target-epoch 2021-11-29 --jitter --range-scaling 1.0 --range-rate-scaling 0.2  -c && ./beampark_correlator.py eiscat_esr /home/danielk/git/antisat_analysis/projects/output/r

In [71]:
for key in kosmos_correlation_select:
    paths = []
    for pth in kosmos_correlation_select[key]:
        if pth is None:
            paths.append(None)
            continue
        if not pth.is_file():
            paths.append(list(pth.glob('*.npy'))[0])
        else:
            paths.append(pth)
    kosmos_correlation_select[key] = paths